<a href="https://colab.research.google.com/github/ns-it/FL_Xray_Flower/blob/main/Path2_MobileNetV2_Experiment_Sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q tensorflow
!pip install -q flwr[simulation] matplotlib numpy pandas kaggle

In [ ]:

# --- CONFIG: centralize hyperparameters here ---
# NUM_ROUNDS defined in CONFIG cell above
NUM_ROUNDS = 15
NUM_CLIENTS = 10
MIN_CLIENTS = int(NUM_CLIENTS * 0.8)
BATCH_SIZE = 32
LOCAL_EPOCHS_PER_ROUND = 5         # local epochs per round (E)
CLIENT_LR = 0.0005                 # learning rate used on clients (SGD)
SERVER_ETA = 0.005                 # server optimizer eta (FedAdam)
SERVER_BETA1 = 0.9
SERVER_BETA2 = 0.999
PROXIMAL_MU = 0.01                 # recommended FedProx mu (try 0.001,0.01,0.1 in sweeps)
ALPHA = 0.5  # defined in CONFIG                        # Dirichlet alpha (large => near-IID); change to 0.1/0.5 to create non-IID
SEED = 42
OPTIMIZER = "sgd"
MOMENTUM=0.9
# set random seeds for reproducibility
import numpy as np, random, tensorflow as tf, os
np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)
# End CONFIG


In [ ]:
# --- 1. استيراد المكتبات (نسخة محدثة لحل مشكلة CUDA) ---
import os
import numpy as np
import tensorflow as tf
import flwr as fl
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import files
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import logging

# --- Imports for Transfer Learning ---
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.layers import GlobalAveragePooling2D, Input
from tensorflow.keras.models import Model
# -------------------------------------

# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#  إضافات جديدة لتعريف الأنواع (Types)
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
from flwr.common import NDArrays, Scalar
from typing import Dict, Optional, Tuple

# إخفاء رسائل التحذير الطويلة
logging.getLogger("flwr").setLevel(logging.ERROR)

# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#  التعديل الجديد (الحل الصحيح)
#  بدلاً من إخفاء الـ GPU، سنقوم بضبطه ليسمح "بنمو الذاكرة"
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# 1. إجبار TensorFlow على "نمو الذاكرة"
# هذا يمنع الجلسة الرئيسية من حجز كل الذاكرة، ويترك الباقي لـ Ray
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # اجعل TensorFlow يخصص الذاكرة عند الحاجة فقط (Politely)
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    print(f"--- تم تفعيل نمو الذاكرة (Memory Growth) للـ GPU ---")
  except RuntimeError as e:
    # يجب أن يتم هذا قبل بدء TensorFlow
    print(e)

print(f"TensorFlow Version: {tf.__version__}")
print(f"Flower Version: {fl.__version__}")
print(f"NumPy Version: {np.__version__}")

In [ ]:
# --- 2. إعداد Kaggle وتحميل البيانات ---

# ارفع ملف kaggle.json
if not os.path.exists("/root/.kaggle/kaggle.json"):
    print("يرجى رفع ملف kaggle.json")
    uploaded = files.upload()
    for fn in uploaded.keys():
        !mkdir -p ~/.kaggle
        !mv {fn} ~/.kaggle/
        !chmod 600 ~/.kaggle/kaggle.json
else:
    print("ملف kaggle.json موجود بالفعل.")

In [ ]:
# --- 3. تحميل وفك ضغط البيانات ---
if not os.path.exists("chest_xray"):
    print("تحميل وفك ضغط مجموعة البيانات...")
    !kaggle datasets download -d paultimothymooney/chest-xray-pneumonia
    !unzip -q chest-xray-pneumonia.zip
    print("تم فك ضغط البيانات.")
else:
    print("مجلد البيانات chest_xray موجود بالفعل.")

In [ ]:
# --- 4. تجهيز البيانات (Data Preprocessing) ---
# --- (نسخة معدلة لـ MobileNetV2) ---

# استيراد الدالة الخاصة بالتهيئة
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# تحديد مسارات المجلدات
base_dir = 'chest_xray'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

# مسارات فئات التدريب
train_normal_dir = os.path.join(train_dir, 'NORMAL')
train_pneumonia_dir = os.path.join(train_dir, 'PNEUMONIA')

num_normal = len(os.listdir(train_normal_dir))
num_pneumonia = len(os.listdir(train_pneumonia_dir))
print(f"Number of (NORMAL) training Images: {num_normal}")
print(f"Number of (PNEUMONIA) training Images: {num_pneumonia}")

# تحديد الثوابت
IMG_SIZE = (128, 128)
BATCH_SIZE = 32 # (موجودة في خلية الإعدادات أصلاً)


train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input, # <--- تعديل هنا
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.15,
    height_shift_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True,
    shear_range=0.1
)

# مولد بيانات التحقق والاختبار (فقط تهيئة)
val_test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# مولد التدريب (80% من بيانات train_dir)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training',
    color_mode='rgb'
)

validation_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2
).flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation',
    color_mode='rgb',
    shuffle=False
)

# مولد الاختبار (من test_dir)
test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    color_mode='rgb',
    shuffle=False
)

---

##  معالجة عدم توازن الفئات (Class Imbalance)

نلاحظ أن عدد صور الالتهاب الرئوي (Pneumonia) أكبر بكثير من الصور الطبيعية (Normal). سنقوم بحساب "أوزان الفئات" (Class Weights) لإجبار النموذج على إعطاء "عقوبة" أكبر عند الخطأ في تصنيف الفئة الأقل (Normal).

In [ ]:
# --- 5. حساب أوزان الفئات ---
class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weight_dict = dict(enumerate(class_weights))
print(f"Class Weights (أوزان الفئات): {class_weight_dict}")

---

## بناء نموذج CNN

سنقوم بتعريف دالة لإنشاء نموذج CNN بسيط.

In [ ]:
# --- 6. بناء النموذج (Model Definition) ---
# --- (نسخة معدلة لاستخدام MobileNetV2 و Transfer Learning) ---

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import SGD, Adam

# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#  التعديل: تعريف دالة بناء النموذج الجديدة
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
def create_cnn_model():
    # 1. تحديد مدخلات النموذج (يجب أن تكون 3 قنوات لـ MobileNetV2)
    inputs = Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))

    # 2. تحميل نموذج MobileNetV2
    # include_top=False: لإزالة طبقة التصنيف الأصلية (1000 فئة)
    # weights='imagenet': لتحميل الأوزان التي تعلمها من ImageNet
    base_model = MobileNetV2(
        include_top=False,
        weights='imagenet',
        input_tensor=inputs
    )

    # 3. تجميد النموذج الأساسي (أهم خطوة في Transfer Learning)
    # لن يتم تحديث أوزان MobileNetV2 أثناء التدريب
    base_model.trainable = False

    # 4. إضافة طبقات التصنيف الخاصة بنا
    x = base_model.output
    x = GlobalAveragePooling2D()(x) # <--- لتقليل الأبعاد
    x = Dense(128, activation='relu')(x) # <--- طبقة كثيفة لنتعلم منها
    x = Dropout(0.5)(x)                  # <--- لتقليل Overfitting
    outputs = Dense(1, activation='sigmoid')(x) # <--- طبقة المخرجات

    # 5. بناء النموذج النهائي
    model = Model(inputs=inputs, outputs=outputs)

    # 6. تحديد المُحسِّن (Optimizer)
    # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    #  هام: عند استخدام Transfer Learning، نبدأ بمعدل تعلم (LR) أقل
    # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

    # سنستخدم 0.001 بدلاً من 0.01 (المعرفة في خلية الإعدادات)
    # هذا يمنع تدمير الأوزان الجيدة التي جاءت من ImageNet
    transfer_learning_lr = CLIENT_LR

    if (OPTIMIZER == "sgd"):
        optimizer = SGD(learning_rate=transfer_learning_lr, momentum=MOMENTUM, clipnorm=1.0)
    elif (OPTIMIZER == "adam"):
        optimizer = Adam(learning_rate=transfer_learning_lr, clipnorm=1.0)

    # 7. تجميع النموذج (Compile)
    model.compile(optimizer=optimizer,
                  loss=BinaryCrossentropy(name='loss'),
                  metrics=['accuracy',
                           Precision(name='precision'),
                           Recall(name='recall'),
                           AUC(name='auc')])
    return model

---

##  التدريب المركزي (Centralized Training)

الآن، سنقوم بتدريب النموذج بالطريقة التقليدية (المركزية) باستخدام جميع بيانات التدريب. سنستخدم `class_weight_dict` الذي حسبناه.

In [ ]:
import os
import pickle
from tensorflow.keras.callbacks import EarlyStopping

backup_file = 'centralized_training_backup.pkl'

centralized_model = create_cnn_model()
centralized_model.summary()

early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
)

print("\n--- Starting Centralized Model Training (with Early Stopping) ---")

history = centralized_model.fit(
        train_generator,
        epochs=50,
        validation_data=validation_generator,
        class_weight=class_weight_dict,
        callbacks=[early_stopping]
)


print("\n--- Evaluating Centralized Model (Best Version) ---")
results = centralized_model.evaluate(test_generator, return_dict=True)
loss = results['loss']
accuracy = results['accuracy']
precision = results.get('precision', 0.0)
recall = results.get('recall', 0.0)
auc = results.get('auc', 0.0)

print(f"Centralized Learning Loss: {loss:.4f}")
print(f"Centralized Learning Accuracy: {accuracy:.4f}")
print(f"Centralized Learning Precision: {precision:.4f}")
print(f"Centralized Learning Recall: {recall:.4f}")
print(f"Centralized Learning AUC: {auc:.4f}")


centralized_results = results


centralized_history_dict = history.history

data_to_save = {
        'history_dict': centralized_history_dict,
        'results_dict': centralized_results
}
with open(backup_file, 'wb') as f:
        pickle.dump(data_to_save, f)
print(f"--- ✅ تم حفظ نتائج التدريب المركزي في {backup_file} ---")

---

## الخطوة 8: تجهيز البيانات للتعلم الاتحادي (FL)

لمحاكاة التعلم الاتحادي، نحتاج إلى تحميل البيانات كـ `Numpy arrays` ثم تقسيمها. سنقوم **بخلط (Shuffle)** البيانات قبل التقسيم لضمان أن كل عميل يحصل على مزيج من الفئتين (IID).

In [ ]:
# --- 8. إعداد التعلم الاتحادي (FL Setup - محاكاة Non-IID واقعي) ---
# --- (نسخة معدلة لـ MobileNetV2) ---

from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# دالة لتحميل البيانات كـ NumPy arrays (تبقى كما هي)
def load_data_as_numpy(generator):
    images, labels = next(generator)
    return images, labels

# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#  التعديل: استخدام دالة التهيئة و 'rgb'
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# مولد لتحميل كل بيانات التدريب (80% split) مرة واحدة
# (يجب أن نستخدم train_generator.n الذي تم حسابه في الخلية 8)
train_full_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input, # <--- تعديل هنا
    validation_split=0.2
).flow_from_directory(
    train_dir, target_size=IMG_SIZE,
    batch_size=train_generator.n,
    class_mode='binary',
    subset='training',
    color_mode='rgb', # <--- تعديل هنا
    shuffle=False
)

# مولد لتحميل كل بيانات الاختبار مرة واحدة
# (يجب أن نستخدم test_generator.n الذي تم حسابه في الخلية 8)
test_full_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input # <--- تعديل هنا
).flow_from_directory(
    test_dir, target_size=IMG_SIZE,
    batch_size=test_generator.n,
    class_mode='binary',
    color_mode='rgb', # <--- تعديل هنا
    shuffle=False
)

print("\nLoading data for FL simulation...")
x_train_fl, y_train_fl = load_data_as_numpy(train_full_generator)
x_test_fl, y_test_fl = load_data_as_numpy(test_full_generator)

print(f"FL training data shape: {x_train_fl.shape}")
print(f"FL training labels shape: {y_train_fl.shape}")
print(f"FL test data shape: {x_test_fl.shape}")
print(f"FL test labels shape: {y_test_fl.shape}")


# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#  الجزء الخاص بتوزيع ديريخليه (Dirichlet)
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

NUM_CLIENTS = 10 # (معرف في خلية الإعدادات)
NUM_CLASSES = 2 # (0: NORMAL, 1: PNEUMONIA)
ALPHA = ALPHA  # (معرف في خلية الإعدادات)

print(f"\nPartitioning data into {NUM_CLIENTS} clients using Dirichlet (Alpha={ALPHA})...")

# 1. فصل مؤشرات (indices) كل فئة
class_indices = [np.where(y_train_fl == i)[0] for i in range(NUM_CLASSES)]

# 2. إنشاء قوائم فارغة لكل عميل
client_data_indices = [[] for _ in range(NUM_CLIENTS)]

# 3. توزيع مؤشرات كل فئة على حدة
for class_idx in range(NUM_CLASSES):
    indices_for_class = class_indices[class_idx]
    num_samples_in_class = len(indices_for_class)

    # خلط المؤشرات داخل الفئة
    np.random.shuffle(indices_for_class)

    # إنشاء توزيع ديريخليه
    proportions = np.random.dirichlet([ALPHA] * NUM_CLIENTS)

    # حساب عدد العينات لكل عميل
    client_samples_per_class = (proportions * num_samples_in_class).astype(int)

    # معالجة البواقي
    remainder = num_samples_in_class - client_samples_per_class.sum()
    client_samples_per_class[np.argmax(proportions)] += remainder

    # 4. تقسيم المؤشرات
    start = 0
    for client_id in range(NUM_CLIENTS):
        num_samples = client_samples_per_class[client_id]
        end = start + num_samples
        client_data_indices[client_id].extend(indices_for_class[start:end])
        start = end

# 5. إنشاء بيانات العميل النهائية
client_data = []
print("\n--- Client Data Distribution (Non-IID) ---")
for client_id in range(NUM_CLIENTS):
    client_indices = client_data_indices[client_id]
    np.random.shuffle(client_indices)

    client_x = x_train_fl[client_indices]
    client_y = y_train_fl[client_indices]

    client_data.append((client_x, client_y))

    if client_id < 10: # طباعة كل العملاء
        num_normal = np.sum(client_y == 0)
        num_pneumonia = np.sum(client_y == 1)
        print(f"Client {client_id}: Total={len(client_y)} | NORMAL={num_normal} | PNEUMONIA={num_pneumonia}")

print("...")
print(f"Data partitioned on: {len(client_data)} Clients (Realistic Non-IID).")

In [ ]:
# ---  عرض توزيع البيانات (Data Distribution Visualization) ---

# 1. تجميع الإحصائيات من 'client_data'
# (نفترض أن 0 = NORMAL, 1 = PNEUMONIA)
distribution_data = []
for client_id in range(NUM_CLIENTS):
    client_y = client_data[client_id][1]
    num_normal = np.sum(client_y == 0)
    num_pneumonia = np.sum(client_y == 1)
    distribution_data.append({
        "Client": f"C{client_id}",
        "NORMAL": num_normal,
        "PNEUMONIA": num_pneumonia
    })

# 2. تحويلها إلى DataFrame لسهولة الرسم
df_dist = pd.DataFrame(distribution_data)

# 3. إعداد بيانات الرسم البياني المكدس (Stacked Bar Chart)
client_labels = df_dist["Client"]
normal_counts = df_dist["NORMAL"]
pneumonia_counts = df_dist["PNEUMONIA"]

# 4. إنشاء الرسم البياني
plt.figure(figsize=(12, 6))
bar_width = 0.7

# رسم الفئة الأولى (NORMAL)
plt.bar(client_labels, normal_counts, bar_width, label='NORMAL (Class 0)', color='blue')

# رسم الفئة الثانية (PNEUMONIA) فوقها
plt.bar(client_labels, pneumonia_counts, bar_width, bottom=normal_counts, label='PNEUMONIA (Class 1)', color='orange')

plt.title(f'Data Distribution per Client (ALPHA = {ALPHA})')
plt.xlabel('Client ID')
plt.ylabel('Number of Samples')
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)

# إضافة عدد العينات الكلي فوق كل عمود
totals = normal_counts + pneumonia_counts
for i, total in enumerate(totals):
    plt.text(i, total + 5, str(total), ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

---
 تعريف العميل الاتحادي (FL Client)

سنقوم بتعريف `class` يمثل العميل. كل عميل سيقوم بـ:
1.  استلام النموذج العام (Global Model).
2.  تقسيم بياناته المحلية إلى تدريب وتحقق (80/20).
3.  حساب أوزان الفئات (Class Weights) *المحلية* الخاصة به.
4.  تطبيق زيادة البيانات (Augmentation) *أثناء* التدريب المحلي.
5.  التدريب لـ 3 دورات (Epochs) محلياً.
6.  إرسال الأوزان المحدثة.

In [ ]:
# --- 9. تعريف العميل الاتحادي (FL Client Definition) ---
# --- (نسخة مُصححة من الحلقة اللانهائية) ---
import math # <--- إضافة لاستخدام math.ceil

class XRayClient(fl.client.NumPyClient):

    def __init__(self, model, x_client, y_client, client_id):
        self.model = model
        self.client_id = client_id

        # مولد بيانات للتدريب المحلي مع زيادة البيانات
        self.train_datagen = ImageDataGenerator(
            # preprocessing_function=preprocess_input, # <--- تعديل هام جداً
            rotation_range=15,
            width_shift_range=0.1,
            height_shift_range=0.1,
            zoom_range=0.1,
            horizontal_flip=True
        )

        # التحقق من عدد العينات قبل التقسيم (من الإصلاح السابق)
        if len(y_client) < 2:
            print(f"[Client {self.client_id}]: Warning: Received only {len(y_client)} samples. Using all for training.")
            self.x_train, self.y_train = x_client, y_client
            self.x_val, self.y_val = x_client[0:0], y_client[0:0]
        else:
            try:
                self.x_train, self.x_val, self.y_train, self.y_val = train_test_split(
                    x_client, y_client, test_size=0.2, random_state=42, stratify=y_client
                )
            except ValueError:
                print(f"[Client {self.client_id}]: Warning: Stratify failed. Using regular split.")
                self.x_train, self.x_val, self.y_train, self.y_val = train_test_split(
                    x_client, y_client, test_size=0.2, random_state=42
                )

    @tf.function
    def _train_step(self, x_batch, y_batch, global_weights_tensors, mu, sample_weights_tensor):
        with tf.GradientTape() as tape:
            y_pred = self.model(x_batch, training=True)
            loss = self.model.loss(y_batch, y_pred)

            if sample_weights_tensor is not None:
                loss = loss * sample_weights_tensor
                loss = tf.reduce_mean(loss)

            if mu > 0.0:
                prox_term = 0.0
                for model_weight, global_weight in zip(self.model.trainable_weights, global_weights_tensors):
                    prox_term += tf.reduce_sum(tf.square(model_weight - global_weight))
                loss += (mu / 2) * prox_term

        grads = tape.gradient(loss, self.model.trainable_weights)
        self.model.optimizer.apply_gradients(zip(grads, self.model.trainable_weights))
        return loss


    def get_parameters(self, config):
        return self.model.get_weights()


    def fit(self, parameters, config):

        print(f"\n--- [Client {self.client_id}]: Starting fit() for Round ---")

        self.model.set_weights(parameters)
        global_weights_tensors = [tf.convert_to_tensor(w) for w in parameters]
        mu = config.get("mu", 0.0)

        if mu > 0.0:
            print(f"[Client {self.client_id}]: Mode = FedProx (mu={mu})")
        else:
            print(f"[Client {self.client_id}]: Mode = FedAvg (mu=0.0)")

        # (حساب أوزان الفئات)
        local_weight_dict = None
        if len(np.unique(self.y_train)) > 1:
            try:
                local_weights = compute_class_weight('balanced', classes=np.unique(self.y_train), y=self.y_train)
                local_weight_dict = dict(enumerate(local_weights))
            except ValueError:
                pass

        # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        #  التعديل: حساب steps_per_epoch
        # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        # BATCH_SIZE تم تعريفها في الخلية 4 (كانت 32)
        steps_per_epoch = math.ceil(len(self.x_train) / BATCH_SIZE)

        # 2. حلقة التدريب الخارجية (بايثون)
        for epoch in range(LOCAL_EPOCHS_PER_ROUND):
            print(f"[Client {self.client_id}]: Starting local epoch {epoch+1}/{LOCAL_EPOCHS_PER_ROUND}...")
            train_flow = self.train_datagen.flow(
                self.x_train, self.y_train, batch_size=BATCH_SIZE, shuffle=True, seed = 42
            )

            batch_count = 0
            for x_batch, y_batch in train_flow:
                # (حساب أوزان العينات)
                sample_weights_tensor = None
                if local_weight_dict:
                    sample_weights_np = np.array([local_weight_dict[c] for c in y_batch])
                    sample_weights_tensor = tf.convert_to_tensor(sample_weights_np, dtype=tf.float32)

                loss_value = self._train_step(x_batch, y_batch, global_weights_tensors, mu, sample_weights_tensor)

                if batch_count % 10 == 0:
                    print(f"[Client {self.client_id}]: Epoch {epoch+1}, Batch {batch_count}/{steps_per_epoch}, Loss: {loss_value.numpy():.4f}...")

                batch_count += 1

                # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
                #  التعديل: إضافة شرط الخروج من الحلقة
                # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
                if batch_count >= steps_per_epoch:
                    break # <--- الخروج من الحلقة الداخلية

            print(f"[Client {self.client_id}]: === Finished local epoch {epoch+1}/{LOCAL_EPOCHS_PER_ROUND} ===")

        # تقييم الأداء
        val_metrics = self.model.evaluate(self.x_val, self.y_val, verbose=0, return_dict=True)
        print(f"--- [Client {self.client_id}]: Finished fit(). Local val_acc: {val_metrics.get('accuracy', 0.0):.4f} ---")
        return self.model.get_weights(), len(self.x_train), {"local_accuracy": val_metrics.get('accuracy', 0.0)}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        metrics = self.model.evaluate(x_test_fl, y_test_fl, verbose=0, return_dict=True)
        return metrics['loss'], len(x_test_fl), metrics

---

## الخطوة 10: تعريف استراتيجية الخادم (Server Strategy)

سنقوم بتعريف دالة التقييم (`evaluate_on_server`) التي سيستخدمها الخادم لتقييم النموذج المُجمّع (Aggregated Model) في نهاية كل جولة باستخدام بيانات الاختبار العامة (`x_test_fl`).

سنستخدم استراتيجية `FedAvg` (Federated Averaging) لتجميع الأوزان.

In [ ]:
# --- 10. تعريف استراتيجيات الخادم ---
from flwr.common import Context
from flwr.server.strategy import FedAdam
from flwr.common import ndarrays_to_parameters
from flwr.common import NDArrays, Scalar
from typing import Dict, Optional, Tuple


# دالة تقييم على الخادم (محدثة لجمع كل المقاييس)
def evaluate_on_server(server_round: int, parameters: NDArrays, config: Dict[str, Scalar]) -> Optional[Tuple[float, Dict[str, Scalar]]]:
    model = create_cnn_model()
    model.set_weights(parameters)

    metrics = model.evaluate(x_test_fl, y_test_fl, verbose=0, return_dict=True)

    print(f"\n=======================================================")
    print(f"ROUND {server_round}: Aggregated Accuracy = {metrics['accuracy']:.4f}")
    print(f" (Loss: {metrics['loss']:.4f}, Prec: {metrics.get('precision', 0.0):.4f}, Rec: {metrics.get('recall', 0.0):.4f})")
    print(f"=======================================================\n")

    return metrics['loss'], metrics


# دالة إنشاء العميل (باستخدام التعريف الأصلي cid: str)
def client_fn(cid: str) -> XRayClient:
    model = create_cnn_model()
    client_x, client_y = client_data[int(cid)]
    return XRayClient(model, client_x, client_y, client_id=cid)


MIN_CLIENTS = int(NUM_CLIENTS * 0.8)

# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#  التعديل: تعريف الأوزان الأولية مرة واحدة للجميع
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
print("--- Creating initial model for ALL strategies ---")
model_for_init = create_cnn_model()
initial_params = ndarrays_to_parameters(model_for_init.get_weights())
print("--- Initial parameters created successfully ---")

# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#  الإضافة: تقييم النموذج الأولي على بيانات التحقق
#  (لنرسم نقطة البداية "Epoch 0" للنموذج المركزي)
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
print("--- Evaluating initial model on validation set (for Epoch 0 plot) ---")
# نستخدم 'validation_generator' لأن 'history' (المركزي) يستخدمها
# نستخدم 'return_dict=True' لسهولة القراءة
initial_model_val_metrics = model_for_init.evaluate(validation_generator, return_dict=True, verbose=0)
print(f"--- Initial val_accuracy (Epoch 0): {initial_model_val_metrics['accuracy']:.4f} ---")

# 1. استراتيجية FedAvg العادية (مدمجة)
strategy_fedavg = fl.server.strategy.FedAvg(
    initial_parameters=initial_params,
    fraction_fit=1.0,
    min_fit_clients=MIN_CLIENTS,
    min_available_clients=NUM_CLIENTS,
    evaluate_fn=evaluate_on_server,
    fraction_evaluate=0.0
)

# 2. استراتيجية FedProx (المدمجة)
strategy_fedprox = fl.server.strategy.FedProx(
    initial_parameters=initial_params,
    fraction_fit=1.0,
    min_fit_clients=MIN_CLIENTS,
    min_available_clients=NUM_CLIENTS,
    evaluate_fn=evaluate_on_server,
    # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    #  التعديل: زيادة "قوة" العقوبة 10x
    # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    proximal_mu=PROXIMAL_MU,
    fraction_evaluate=0.0
)

# 3. استراتيجية FedAdam (المدمجة)
strategy_fedadam = FedAdam(
    initial_parameters=initial_params,
    fraction_fit=1.0,
    min_fit_clients=MIN_CLIENTS,
    min_available_clients=NUM_CLIENTS,
    evaluate_fn=evaluate_on_server,
    eta=SERVER_ETA,
    beta_1=SERVER_BETA1,
    beta_2=SERVER_BETA2,
    tau=1e-7,
    fraction_evaluate=0.0
)

---

## الخطوة 11: بدء محاكاة التعلم الاتحادي (FL Simulation)

الآن، نبدأ المحاكاة لـ 15 جولة اتحادية (Federated Rounds).

In [ ]:
# --- 11. بدء المحاكاة (لكل الاستراتيجيات) ---

# NUM_ROUNDS defined in CONFIG cell above

# إعدادات Ray لاستخدام الـ GPU
ray_init_args = {
    "include_dashboard": False,
    "num_gpus": 1,
    "num_cpus": 2    # <--- نضيف هذا لتحديد التوازي
}

# إخبار Flower بإعطاء 10% من الـ GPU لكل عميل
client_resources = {
    "num_gpus": 0.1,

}

print(f"Ray init args: {ray_init_args}")

In [ ]:
# 1. تشغيل FedAvg
print("\n--- بدء محاكاة (FedAvg) ---")
history_fedavg = fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=NUM_ROUNDS),
    strategy=strategy_fedavg,
    ray_init_args=ray_init_args,
    client_resources=client_resources
)


In [ ]:
# 2. تشغيل FedProx
print("\n--- بدء محاكاة (FedProx) ---")
history_fedprox = fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=NUM_ROUNDS),
    strategy=strategy_fedprox,
    ray_init_args=ray_init_args,
    client_resources=client_resources
)

In [ ]:
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#  إضافة جديدة: تشغيل FedAdam
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
print("\n--- بدء محاكاة (FedAdam) ---")
history_fedadam = fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=NUM_ROUNDS),
    strategy=strategy_fedadam,
    ray_init_args=ray_init_args,
    client_resources=client_resources
)

---

## الخطوة 12: المقارنة النهائية

أخيراً، نقوم بجمع النتائج من التدريب المركزي والتدريب الاتحادي ونعرضها في جدول ورسم بياني للمقارنة.

In [ ]:
# --- 12. المقارنة النهائية (مقارنة 4 نماذج مع AUC) ---

# (جلب مقاييس "Epoch 0" المركزية - يبقى كما هو)
init_metrics = initial_model_val_metrics

# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#  التعديل: القراءة من القاموس الذي تم تحميله
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# بدلاً من history.history، نستخدم centralized_history_dict
# (الذي تم تعريفه في الخلية 7، سواء بالتدريب أو بالتحميل)
cen_epochs_hist = range(1, len(centralized_history_dict['val_accuracy']) + 1)
cen_acc_hist = centralized_history_dict['val_accuracy']
cen_loss_hist = centralized_history_dict['val_loss']
cen_prec_hist = centralized_history_dict['val_precision']
cen_rec_hist = centralized_history_dict['val_recall']
cen_auc_hist = centralized_history_dict['val_auc']


# (دمج "Epoch 0" مع باقي الدورات - يبقى كما هو)
cen_epochs_plot = [0] + list(cen_epochs_hist)
cen_acc_plot = [init_metrics['accuracy']] + cen_acc_hist
cen_loss_plot = [init_metrics['loss']] + cen_loss_hist
cen_prec_plot = [init_metrics.get('precision', 0.0)] + cen_prec_hist
cen_rec_plot = [init_metrics.get('recall', 0.0)] + cen_rec_hist
cen_auc_plot = [init_metrics.get('auc', 0.0)] + cen_auc_hist


# (دالة استخراج المقاييس الاتحادية - تبقى كما هي)
def extract_fl_history(fl_history, local_epochs_per_round):
    loss_hist = fl_history.losses_centralized
    metrics_hist = fl_history.metrics_centralized
    x_axis = [(r * local_epochs_per_round) for r, _ in loss_hist]
    loss = [l for r, l in loss_hist]
    acc = [m for r, m in metrics_hist.get('accuracy', [])]
    prec = [m for r, m in metrics_hist.get('precision', [])]
    rec = [m for r, m in metrics_hist.get('recall', [])]
    auc = [m for r, m in metrics_hist.get('auc', [])]
    return x_axis, acc, loss, prec, rec, auc


# (تحديد الدورات المحلية - يبقى كما هو)
# LOCAL_EPOCHS_PER_ROUND = 3

# (استخراج بيانات النماذج الاتحادية - يبقى كما هو)
x_fedavg, acc_fedavg, loss_fedavg, prec_fedavg, rec_fedavg, auc_fedavg = extract_fl_history(history_fedavg, LOCAL_EPOCHS_PER_ROUND)
x_fedprox, acc_fedprox, loss_fedprox, prec_fedprox, rec_fedprox, auc_fedprox = extract_fl_history(history_fedprox, LOCAL_EPOCHS_PER_ROUND)
x_fedadam, acc_fedadam, loss_fedadam, prec_fedadam, rec_fedadam, auc_fedadam = extract_fl_history(history_fedadam, LOCAL_EPOCHS_PER_ROUND)


# (كود الرسم البياني - يبقى كما هو تماماً)
# ... (كل كود plt.subplot) ...
plt.figure(figsize=(16, 18))

# المخطط 1: الدقة (Accuracy)
plt.subplot(3, 2, 1)
plt.plot(cen_epochs_plot, cen_acc_plot, label='Centralized (Validation)', linestyle='--', color='red')
plt.plot(x_fedavg, acc_fedavg, label='FedAvg', marker='o', color='blue', markersize=4)
plt.plot(x_fedprox, acc_fedprox, label=f'FedProx (mu={PROXIMAL_MU})', marker='x', color='green', markersize=5)
plt.plot(x_fedadam, acc_fedadam, label=f'FedAdam (eta={SERVER_ETA})', marker='s', color='purple', markersize=4)
plt.title('Accuracy Progression')
plt.xlabel('Equivalent Centralized Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

# المخطط 2: الخسارة (Loss)
plt.subplot(3, 2, 2)
plt.plot(cen_epochs_plot, cen_loss_plot, label='Centralized (Validation)', linestyle='--', color='red')
plt.plot(x_fedavg, loss_fedavg, label='FedAvg', marker='o', color='blue', markersize=4)
plt.plot(x_fedprox, loss_fedprox, label=f'FedProx (mu={PROXIMAL_MU})', marker='x', color='green', markersize=5)
plt.plot(x_fedadam, loss_fedadam, label=f'FedAdam (eta={SERVER_ETA})', marker='s', color='purple', markersize=4)
plt.title('Loss Progression')
plt.xlabel('Equivalent Centralized Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

# المخطط 3: الدقة (Precision)
plt.subplot(3, 2, 3)
plt.plot(cen_epochs_plot, cen_prec_plot, label='Centralized (Validation)', linestyle='--', color='red')
plt.plot(x_fedavg, prec_fedavg, label='FedAvg', marker='o', color='blue', markersize=4)
plt.plot(x_fedprox, prec_fedprox, label=f'FedProx (mu={PROXIMAL_MU})', marker='x', color='green', markersize=5)
plt.plot(x_fedadam, prec_fedadam, label=f'FedAdam (eta={SERVER_ETA})', marker='s', color='purple', markersize=4)
plt.title('Precision Progression')
plt.xlabel('Equivalent Centralized Epochs')
plt.ylabel('Precision')
plt.legend()
plt.grid(True)

# المخطط 4: الاستدعاء (Recall)
plt.subplot(3, 2, 4)
plt.plot(cen_epochs_plot, cen_rec_plot, label='Centralized (Validation)', linestyle='--', color='red')
plt.plot(x_fedavg, rec_fedavg, label='FedAvg', marker='o', color='blue', markersize=4)
plt.plot(x_fedprox, rec_fedprox, label=f'FedProx (mu={PROXIMAL_MU})', marker='x', color='green', markersize=5)
plt.plot(x_fedadam, rec_fedadam, label=f'FedAdam (eta={SERVER_ETA})', marker='s', color='purple', markersize=4)
plt.title('Recall Progression')
plt.xlabel('Equivalent Centralized Epochs')
plt.ylabel('Recall')
plt.legend()
plt.grid(True)

# المخطط 5: AUC
plt.subplot(3, 2, 5)
plt.plot(cen_epochs_plot, cen_auc_plot, label='Centralized (Validation)', linestyle='--', color='red')
plt.plot(x_fedavg, auc_fedavg, label='FedAvg', marker='o', color='blue', markersize=4)
plt.plot(x_fedprox, auc_fedprox, label=f'FedProx (mu={PROXIMAL_MU})', marker='x', color='green', markersize=5)
plt.plot(x_fedadam, auc_fedadam, label=f'FedAdam (eta={SERVER_ETA})', marker='s', color='purple', markersize=4)
plt.title('AUC Progression')
plt.xlabel('Equivalent Centralized Epochs')
plt.ylabel('AUC')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# --- 13. جدول ملخص النتائج النهائية ---

print("\n" + "="*60)
print("--- جدول ملخص النتائج النهائية (القيم النهائية من الجولة الأخيرة) ---")
print("="*60 + "\n")

# دالة آمنة لجلب القيمة الأخيرة من قائمة (لتجنب الأخطاء إذا كانت فارغة)
def get_final_metric(metric_list):
    try:
        # إرجاع العنصر الأخير في القائمة
        return metric_list[-1]
    except (IndexError, TypeError):
        # إرجاع 0.0 إذا كانت القائمة فارغة
        return 0.0

# 1. جلب بيانات النموذج المركزي
# (من المتغير 'centralized_results' الذي خزنناه في الخلية 7)
cen_metrics = {
    'Accuracy': centralized_results.get('accuracy', 0.0),
    'Loss': centralized_results.get('loss', 0.0),
    'Precision': centralized_results.get('precision', 0.0),
    'Recall': centralized_results.get('recall', 0.0),
    'AUC': centralized_results.get('auc', 0.0)
}

# 2. جلب البيانات النهائية للنماذج الاتحادية
# (من القوائم التي استخرجناها في الخلية 12)
fedavg_metrics = {
    'Accuracy': get_final_metric(acc_fedavg),
    'Loss': get_final_metric(loss_fedavg),
    'Precision': get_final_metric(prec_fedavg),
    'Recall': get_final_metric(rec_fedavg),
    'AUC': get_final_metric(auc_fedavg)
}

fedprox_metrics = {
    'Accuracy': get_final_metric(acc_fedprox),
    'Loss': get_final_metric(loss_fedprox),
    'Precision': get_final_metric(prec_fedprox),
    'Recall': get_final_metric(rec_fedprox),
    'AUC': get_final_metric(auc_fedprox)
}

fedadam_metrics = {
    'Accuracy': get_final_metric(acc_fedadam),
    'Loss': get_final_metric(loss_fedadam),
    'Precision': get_final_metric(prec_fedadam),
    'Recall': get_final_metric(rec_fedadam),
    'AUC': get_final_metric(auc_fedadam)
}

# 3. إنشاء قاموس البيانات لـ Pandas
data = {
    "المقياس (Metric)": ["Accuracy", "Loss", "Precision", "Recall", "AUC"],
    "المركزي (Centralized)": [
        cen_metrics['Accuracy'],
        cen_metrics['Loss'],
        cen_metrics['Precision'],
        cen_metrics['Recall'],
        cen_metrics['AUC']
    ],
    "FedAvg": [
        fedavg_metrics['Accuracy'],
        fedavg_metrics['Loss'],
        fedavg_metrics['Precision'],
        fedavg_metrics['Recall'],
        fedavg_metrics['AUC']
    ],
    f"FedProx (mu={PROXIMAL_MU})": [
        fedprox_metrics['Accuracy'],
        fedprox_metrics['Loss'],
        fedprox_metrics['Precision'],
        fedprox_metrics['Recall'],
        fedprox_metrics['AUC']
    ],
    f"FedAdam (eta={SERVER_ETA})": [
        fedadam_metrics['Accuracy'],
        fedadam_metrics['Loss'],
        fedadam_metrics['Precision'],
        fedadam_metrics['Recall'],
        fedadam_metrics['AUC']
    ]
}

# 4. إنشاء وطباعة الجدول
summary_df = pd.DataFrame(data)

# نستخدم .to_string() لضمان طباعة الجدول بالكامل بتنسيق نظيف
print(summary_df.to_string(index=False, float_format='%.4f'))